# TSA Chapter 10: ACF of Squared BTC Returns

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch10/TSA_ch10_btc_acf_squared/TSA_ch10_btc_acf_squared.ipynb)

This notebook demonstrates:
- Computing the autocorrelation function of squared Bitcoin returns to detect volatility clustering (ARCH effects).

In [ ]:
!pip install yfinance statsmodels matplotlib numpy pandas -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import yfinance as yf
from statsmodels.tsa.stattools import acf
import os, warnings
warnings.filterwarnings('ignore')

In [ ]:
# Color scheme and style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12,
    'axes.labelsize': 10, 'xtick.labelsize': 9, 'ytick.labelsize': 9,
    'legend.fontsize': 9, 'figure.dpi': 150, 'lines.linewidth': 1.2,
    'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    try:
        charts_path = os.path.join('..', '..', '..', 'charts', name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass
    print(f'Saved: {name}.pdf + .png')

def legend_outside(ax, ncol=3, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)

In [ ]:
# Download Bitcoin data
btc = yf.download('BTC-USD', start='2019-01-01', progress=False)
if isinstance(btc.columns, pd.MultiIndex):
    btc.columns = btc.columns.get_level_values(0)
btc['Return'] = btc['Close'].pct_change() * 100
btc.dropna(inplace=True)
print(f'BTC data: {len(btc)} daily observations')

# ACF of squared returns
sq = btc['Return']**2
acf_vals, confint = acf(sq, nlags=40, alpha=0.05)
lags = np.arange(len(acf_vals))

fig, ax = plt.subplots(figsize=(7, 3))
ax.bar(lags[1:], acf_vals[1:], width=0.6, color=COLORS['blue'], label='ACF of $r_t^2$')
ci_upper = confint[1:, 1] - acf_vals[1:]
ax.fill_between(lags[1:], -ci_upper, ci_upper, color=COLORS['blue'], alpha=0.15, label='95% CI')
ax.axhline(0, color='black', lw=0.5)
ax.set_xlabel('Lag')
ax.set_ylabel('ACF')
ax.set_title('ACF of Squared BTC Returns')
legend_outside(ax, ncol=2)
fig.tight_layout()
save_chart(fig, 'btc_acf_squared')
plt.show()